In [2]:
import pandas as pd

In [10]:
print('Loading Data')
matched_volumes = pd.read_csv('../input/matched_hathi_estc_nov2023.csv', index_col=0)
estc_data = pd.read_csv('../input/estc_1500_to_1800.csv', index_col=0)
hathitrust_data = pd.read_csv('../temporary/volumes_scores.csv')

Loading Data


C:\Users\Austin\AppData\Local\Temp\ipykernel_10108\2937896531.py:3: DtypeWarning: Columns (29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  estc_data = pd.read_csv('../input/estc_1500_to_1800.csv', index_col=0)


In [17]:
matched_volumes.head()

,Title,Title.1,HTID,ESTC System No.,Match Score
0,A sentimental journey through France and Italy,Yorickâ€™s sentimental journey through France ...,coo.31924096960947,6214773,87.378641
1,Traditions of Eden,The Garden of Eden,coo.31924096961259,6157289,61.111111
2,A history of the county of Brecknock,A list of the court of aldermen,coo.31924096162064,6159287,71.641791
3,"Tree and serpent worship: or, Illustrations of...",The history of the Church of Christ. Volume th...,coo.31924096988872,6059892,56.170213
4,Thirteen satires,The snare,coo.31924096977644,6133567,64.000000


In [15]:
estc_data.head()

,ESTC System No.,ESTC Citation No.,Author - personal,Title,Publisher/year,Physical descr.,General note,Citation/references,Surrogates,Loc. of filmed copy,...,Absorbed by,Continued in pt. by,Related record,Absorbed,Formed by,Changed back to,Continues in part,Geographic coverage,Separated from,Language note
0,6182456,S110939,"Betson, Thomas.",Here begynneth a ryght profytable treatyse co[...,"[Westminster : Wynkyn de Worde, 1500]",[36] p. : ill. ; 4⁰.,"['Caption title.', 'Imprint from STC.', 'Print...","['STC (2nd ed.), 1978', 'Duff, 43', 'GW, 4190']","Available on microfilm. Ann Arbor, Mich.: Univ...",C.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6190320,S118960,NaN,[The history of Guy of Warwick],"[London? : [R. Pynson, 1500?]]",[3]+ leaves ; 4⁰.,['Printer’s name from and publication date con...,"STC (2nd ed.), 12540","Available on microfilm. Ann Arbor, Mich.: Univ...",L.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6203003,S93554,NaN,[Horae ad usum Sarum.],[Thys emprynteth at Westmynster : by me Julyay...,[32]+ p. ; 64⁰.,"['Title from STC.', 'Imprint from colophon.', ...","STC (2nd ed.), 15895",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6186462,S115128,NaN,[Sir Eglamour],"[Westminster : Wynkyn de Worde, 1500]",[2+] p. ; 4⁰.,['In verse. Second line on only extant leaf re...,"['STC (2nd ed.), 7541', 'Duff, 135']","Available on microfilm. Ann Arbor, Mich.: Univ...",C.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6204328,S92225,NaN,[11 Edw. IV.] De termino sancte Trinitatis [etc.],"London [R. Pynson, 1500?]",[No pagination provided] ; 2⁰.,a-b⁶ c⁴ (a1 blank); not foliated; 16 leaves.,"STC (2nd ed.), 9836",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
pd.set_option('display.max_columns',None)
hathitrust_data.head()

,Religion,Science,Political Economy,HTID,industry_2,industry_3,industry_1643,percent_regression,percent_pessimism,percent_optimistic,percent_progress_original,optimism_score,Unnamed: 0,Year,oclc,Year_rounded,percent_progress_main,percent_progress_secondary,progress_regression_original,progress_regression_main,progress_regression_secondary,optimism_percentile,industry_2_percentile,industry_3_percentile,industry_1643_percentile,optimistic_percentile,progress_percentile_original,progress_percentile_main,progress_percentile_secondary,pessimism_percentile,regression_percentile,progress_regression_percentile_original,progress_regression_percentile_main,progress_regression_percentile_secondary
0,0.269292,0.423439,0.307269,mdp.39015030625571,0.135193,0.137255,0.014212,0.0,0.003202,0.015090,0.003290,0.015177,99079,1745.0,12182269.0,1745.0,0.003290,0.001141,0.003290,0.003290,0.001141,0.881106,0.477442,0.441734,0.411385,0.856724,0.836102,0.880751,0.732677,0.751067,0.490221,0.836376,0.880944,0.733473
1,0.311005,0.335601,0.353394,njp.32101037679113,0.238291,0.268704,0.033303,0.0,0.000456,0.001977,0.003041,0.004562,326,1863.0,2361049.0,1863.0,0.001369,0.000760,0.003041,0.001369,0.000760,0.485528,0.695554,0.706829,0.766916,0.320145,0.818952,0.704572,0.657487,0.367277,0.490221,0.819331,0.705337,0.658581
2,0.323564,0.324238,0.352198,coo1.ark+=13960=t82j70q5m,0.115167,0.140237,0.012692,0.0,0.003229,0.017894,0.001884,0.016549,113419,1812.0,6112636.0,1812.0,0.001884,0.000807,0.001884,0.001884,0.000807,0.902125,0.404073,0.451432,0.354800,0.900559,0.710297,0.769607,0.666897,0.753212,0.490221,0.710856,0.770098,0.667984
3,0.300747,0.359683,0.339569,mdp.39015064485843,0.606536,0.647874,0.070553,0.0,0.000000,0.001550,0.000456,0.002005,96838,1796.0,3572259.0,1796.0,0.000456,0.000456,0.000456,0.000456,0.000456,0.313470,0.901497,0.900521,0.932461,0.290139,0.449692,0.521980,0.605585,0.130762,0.490221,0.451103,0.523422,0.606710
4,0.320944,0.339545,0.339511,hvd.32044102877073,0.399598,0.432432,0.031941,0.0,0.002010,0.015412,0.000000,0.013402,150784,1890.0,83478977.0,1890.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.845555,0.828197,0.828731,0.754131,0.863007,0.165287,0.216157,0.279079,0.628997,0.490221,0.169488,0.221962,0.286382


In [18]:
hathitrust_data.columns

Index(['Religion', 'Science', 'Political Economy', 'HTID', 'industry_2',
       'industry_3', 'industry_1643', 'percent_regression',
       'percent_pessimism', 'percent_optimistic', 'percent_progress_original',
       'optimism_score', 'Unnamed: 0', 'Year', 'oclc', 'Year_rounded',
       'percent_progress_main', 'percent_progress_secondary',
       'progress_regression_original', 'progress_regression_main',
       'progress_regression_secondary', 'optimism_percentile',
       'industry_2_percentile', 'industry_3_percentile',
       'industry_1643_percentile', 'optimistic_percentile',
       'progress_percentile_original', 'progress_percentile_main',
       'progress_percentile_secondary', 'pessimism_percentile',
       'regression_percentile', 'progress_regression_percentile_original',
       'progress_regression_percentile_main',
       'progress_regression_percentile_secondary'],
      dtype='object')

In [22]:
#start by merging year from Hathitrust data
matched_volumes = matched_volumes.merge(hathitrust_data[['HTID', 'Year']], on = 'HTID', how = 'inner')

In [24]:
matched_volumes = matched_volumes[matched_volumes['Year'] <= 1800]

In [25]:
matched_volumes

,Title,Title.1,HTID,ESTC System No.,Match Score,Year
18,Transactions,Instructions,coo.31924003457953,6237091,66.666667,1791.0
31,Vegetable culture for amateurs,Prayers for children,coo.31924003303660,6274137,52.000000,1800.0
84,"Grand Council Royal and Select Masters, Englan...","To Edward Wheler, Esqr. &ca. Council, fort Wil...",coo.31924016286381,6043209,52.252252,1800.0
94,Cucumber culture for amateurs,Prayers for children,coo.31924019147622,6274137,57.142857,1800.0
107,The Law list,The Daily post,coo.31924006955771,6072265,61.538462,1700.0
...,...,...,...,...,...,...
160658,The first book of architecture,The first book of architecture,ucbk.ark+=28722=h29p8v,6152011,100.000000,1721.0
160667,The first book of architecture,The first book of architecture,ucbk.ark+=28722=h20d1n,6152011,100.000000,1716.0
160678,The lives and characters of the English dramat...,An essay on the dramatic character of Sir John...,emu.010002482099,6245451,67.272727,1699.0
160679,"A short account of that part of Africa, inhabi...",A brief account of the present state of the Af...,emu.010002630659,6328251,66.115702,1768.0


In [27]:
len(matched_volumes['HTID'].unique())

18829